## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Carnarvon,AU,-24.8667,113.6333,73.40,71,74,24.23,broken clouds
1,1,Hobart,AU,-42.8794,147.3294,45.00,93,75,6.91,broken clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,51.80,40,20,12.66,few clouds
3,3,Linhares,BR,-19.3911,-40.0722,76.55,70,37,4.56,scattered clouds
4,4,Ballina,AU,-28.8667,153.5667,55.40,82,90,6.91,overcast clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Linhares,BR,-19.3911,-40.0722,76.55,70,37,4.56,scattered clouds
12,12,Rosario,AR,-32.9468,-60.6393,75.20,41,0,6.91,clear sky
14,14,Carauari,BR,-4.8828,-66.8958,84.60,69,54,0.51,broken clouds
22,22,Hilo,US,19.7297,-155.0900,77.00,61,1,10.36,clear sky
23,23,Vaini,TO,-21.2000,-175.2000,78.80,73,20,6.91,few clouds
27,27,Avarua,CK,-21.2078,-159.7750,80.60,78,42,4.61,scattered clouds
38,38,Rikitea,PF,-23.1203,-134.9692,78.49,68,7,10.31,clear sky
40,40,Qeshm,IR,26.9581,56.2719,77.00,83,0,4.61,clear sky
41,41,Georgetown,MY,5.4112,100.3354,80.60,89,20,4.61,few clouds
42,42,Vila Velha,BR,-20.3297,-40.2925,82.40,74,0,8.05,clear sky


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                228
City                   228
Country                224
Lat                    228
Lng                    228
Max Temp               228
Humidity               228
Cloudiness             228
Wind Speed             228
Current Description    228
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City_ID                224
City                   224
Country                224
Lat                    224
Lng                    224
Max Temp               224
Humidity               224
Cloudiness             224
Wind Speed             224
Current Description    224
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Linhares,BR,76.55,scattered clouds,-19.3911,-40.0722,
12,Rosario,AR,75.20,clear sky,-32.9468,-60.6393,
14,Carauari,BR,84.60,broken clouds,-4.8828,-66.8958,
22,Hilo,US,77.00,clear sky,19.7297,-155.0900,
23,Vaini,TO,78.80,few clouds,-21.2000,-175.2000,
27,Avarua,CK,80.60,scattered clouds,-21.2078,-159.7750,
38,Rikitea,PF,78.49,clear sky,-23.1203,-134.9692,
40,Qeshm,IR,77.00,clear sky,26.9581,56.2719,
41,Georgetown,MY,80.60,few clouds,5.4112,100.3354,
42,Vila Velha,BR,82.40,clear sky,-20.3297,-40.2925,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np

hotel_df['Hotel Name'].replace('',np.nan, inplace=True)

clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   211
Country                211
Max Temp               211
Current Description    211
Lat                    211
Lng                    211
Hotel Name             211
dtype: int64

In [28]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))